![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/4bd64c1deb3c4ffb8ca7ff93c1f1497d).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [4]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTest_carListings.zip', index_col=0)

In [5]:
# Visualización datos de entrenamiento
dataTraining.head()

,Price,Year,Mileage,State,Make,Model
0,34995,2017,9913,FL,Jeep,Wrangler
1,37895,2015,20578,OH,Chevrolet,Tahoe4WD
2,18430,2012,83716,TX,BMW,X5AWD
3,24681,2014,28729,OH,Cadillac,SRXLuxury
4,26998,2013,64032,CO,Jeep,Wrangler


In [6]:
# Visualización datos de test
dataTesting.head()

,Year,Mileage,State,Make,Model
ID,,,,,
0,2014,31909,MD,Nissan,MuranoAWD
1,2017,5362,FL,Jeep,Wrangler
2,2014,50300,OH,Ford,FlexLimited
3,2004,132160,WA,BMW,5
4,2015,25226,MA,Jeep,Grand


In [7]:
# Predicción del conjunto de test - acá se genera un número aleatorio como ejemplo
np.random.seed(42)
y_pred = pd.DataFrame(np.random.rand(dataTesting.shape[0]) * 75000 + 5000, index=dataTesting.index, columns=['Price'])

In [8]:
# Guardar predicciones en formato exigido en la competencia de kaggle
y_pred.to_csv('test_submission.csv', index_label='ID')
y_pred.head()

,Price
ID,
0,33090.508914
1,76303.572981
2,59899.545636
3,49899.386315
4,16701.398033


In [9]:
XTotal = dataTraining.loc[:,dataTraining.columns!="Price"]
yTotal = dataTraining.loc[:,dataTraining.columns=="Price"]
XTrain, XTest, yTrain, yTest = train_test_split(XTotal, yTotal, test_size=0.33, random_state=0)

In [10]:
##Buscamos valores Nulos en las variables.
XTrain.isnull().sum()

Year       0
Mileage    0
State      0
Make       0
Model      0
dtype: int64

In [11]:
print(XTrain['Year'].describe())
XTrain['Year'].value_counts()

count    268000.000000
mean       2013.199560
std           3.296668
min        1997.000000
25%        2012.000000
50%        2014.000000
75%        2016.000000
max        2018.000000
Name: Year, dtype: float64


2015    47785
2014    45591
2016    37564
2017    30171
2013    21618
2012    20653
2011    15996
2010    11098
2008     9054
2007     7436
2009     7098
2006     4816
2005     3236
2004     2331
2003     1267
2002      823
2001      512
2000      314
2018      256
1999      195
1998      116
1997       70
Name: Year, dtype: int64

In [12]:
print(XTrain['Mileage'].describe())
XTrain['Mileage'].value_counts()

count    2.680000e+05
mean     5.504151e+04
std      4.066074e+04
min      5.000000e+00
25%      2.582100e+04
50%      4.294000e+04
75%      7.742425e+04
max      1.401471e+06
Name: Mileage, dtype: float64


10        167
5          82
15         60
12         50
11         37
         ... 
124485      1
130630      1
101960      1
110156      1
2047        1
Name: Mileage, Length: 112808, dtype: int64

In [13]:
print(XTrain['State'].describe())
XTrain['State'].value_counts()

count     268000
unique        51
top           TX
freq       30868
Name: State, dtype: object


 TX    30868
 CA    24244
 FL    22681
 GA    12241
 NC    12039
 IL    11379
 VA    10613
 PA     8698
 NY     8409
 NJ     8104
 OH     7560
 WA     7422
 CO     7304
 AZ     7126
 TN     6234
 MA     5770
 MD     5627
 MO     5356
 IN     5230
 KY     4452
 AL     4276
 WI     3726
 MN     3646
 MI     3502
 OK     3393
 CT     3286
 OR     3268
 SC     3213
 UT     3191
 LA     2546
 KS     2503
 NV     2257
 AR     2096
 MS     2013
 NH     1863
 NE     1646
 IA     1574
 NM     1537
 ID     1124
 DE      839
 HI      798
 ME      649
 MT      643
 AK      561
 RI      534
 WV      460
 ND      451
 SD      409
 VT      402
 WY      233
 DC        4
Name: State, dtype: int64

In [14]:
print(XTrain['Make'].describe())
XTrain['Make'].value_counts()

count     268000
unique        38
top         Ford
freq       42123
Name: Make, dtype: object


Ford             42123
Chevrolet        39132
Toyota           30780
Honda            22258
Jeep             16406
GMC              13952
Kia              10994
Dodge            10826
Hyundai          10023
Lexus             9204
BMW               8238
Volkswagen        7463
Nissan            7010
Chrysler          6040
Mercedes-Benz     5108
Subaru            4487
Cadillac          3632
Buick             3245
Ram               2408
MINI              1855
Land              1793
INFINITI          1647
Acura             1603
Mazda             1550
Lincoln           1231
Volvo             1157
Audi               959
Mitsubishi         834
Porsche            655
Scion              449
Jaguar             214
Pontiac            194
FIAT               178
Mercury            173
Tesla              106
Bentley             44
Suzuki              24
Freightliner         5
Name: Make, dtype: int64

In [15]:
print(XTrain['Model'].describe())
XTrain['Model'].value_counts()

count        268000
unique          525
top       Silverado
freq          12126
Name: Model, dtype: object


Silverado          12126
Grand               8360
Sierra              5568
Accord              4890
F-1504WD            4445
                   ...  
RX-84dr               35
Yaris4dr              35
Monte                 33
RegalGS               33
SLK-ClassSLK350       28
Name: Model, Length: 525, dtype: int64

In [16]:
XTrain['YearsBetween']=2022-XTrain['Year']
XTrain['MileagebyYear']=XTrain['Mileage']/XTrain['YearsBetween']
State=pd.get_dummies(XTrain["State"], prefix='State')
XTrain=pd.concat([XTrain, State], axis=1)
Make=pd.get_dummies(XTrain["Make"], prefix='Make')
XTrain=pd.concat([XTrain, Make], axis=1)
Model=pd.get_dummies(XTrain["Model"], prefix='Model')
XTrain=pd.concat([XTrain, Model], axis=1)

In [17]:
#XTrain=XTrain.drop(["State","Make","Model","Make_Freightliner"], axis=1)
XTrain=XTrain.drop(["State","Make","Model"], axis=1)
XTrain

,Year,Mileage,YearsBetween,MileagebyYear,State_ AK,State_ AL,State_ AR,State_ AZ,State_ CA,State_ CO,...,Model_Yaris4dr,Model_YarisBase,Model_YarisLE,Model_Yukon,Model_Yukon2WD,Model_Yukon4WD,Model_Yukon4dr,Model_tC2dr,Model_xB5dr,Model_xD5dr
219421,2012,52027,10,5202.700000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
187898,2015,18127,7,2589.571429,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
298770,2013,122197,9,13577.444444,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
217760,2014,38003,8,4750.375000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
195247,2015,49464,7,7066.285714,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359783,2012,41501,10,4150.100000,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
358083,2010,77127,12,6427.250000,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
152315,2015,34896,7,4985.142857,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
117952,2016,3946,6,657.666667,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
#XTest=dataTesting
XTest['YearsBetween']=2022-XTest['Year']
XTest['MileagebyYear']=XTest['Mileage']/XTest['YearsBetween']
State=pd.get_dummies(XTest["State"], prefix='State')
XTest=pd.concat([XTest, State], axis=1)
Make=pd.get_dummies(XTest["Make"], prefix='Make')
XTest=pd.concat([XTest, Make], axis=1)
Model=pd.get_dummies(XTest["Model"], prefix='Model')
XTest=pd.concat([XTest, Model], axis=1)

In [19]:
XTest=XTest.drop(["State","Make","Model"], axis=1)
XTest

,Year,Mileage,YearsBetween,MileagebyYear,State_ AK,State_ AL,State_ AR,State_ AZ,State_ CA,State_ CO,...,Model_Yaris4dr,Model_YarisBase,Model_YarisLE,Model_Yukon,Model_Yukon2WD,Model_Yukon4WD,Model_Yukon4dr,Model_tC2dr,Model_xB5dr,Model_xD5dr
242160,2010,115991,12,9665.916667,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3413,2014,48093,8,6011.625000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
393024,2016,5494,6,915.666667,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
247987,2016,43217,6,7202.833333,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
52436,2013,37664,9,4184.888889,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239061,2010,167325,12,13943.750000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
237666,2013,63520,9,7057.777778,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40562,2016,32595,6,5432.500000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
136654,2017,5,5,1.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# n_estimatirs calibration
estimator_range = range(50, 400, 25)

accuracy_scores = []
for estimator in estimator_range:
    print(estimator)
    clf = RandomForestRegressor(n_estimators = estimator,max_features='sqrt', random_state = 0)
    clf.fit(XTrain, yTrain)
    yPred= clf.predict(XTest)
    accuracy_scores.append(np.sqrt(metrics.mean_squared_error(yTest, yPred)))    
print(estimator_range[np.argmin(accuracy_scores)])
plt.plot(estimator_range, accuracy_scores)
plt.xlabel('n_estimators')
plt.ylabel('Accuracy')
plt.show()

50
75
100
125
150
175
200
225
250
275


In [ ]:
# max_features calibration
feature_range = range(1, len(XTrain.columns)+1,10)
accuracy_scores = []
for feature in feature_range:
    print(feature)
    clf = RandomForestRegressor(n_estimators=200, max_features='sqrt', random_state=0)
    clf.fit(XTrain, yTrain)
    yPred= clf.predict(XTest)
    accuracy_scores.append(np.sqrt(metrics.mean_squared_error(yTest, yPred)))   
plt.plot(feature_range, accuracy_scores)
plt.xlabel('max_features')
plt.ylabel('Accuracy')
plt.show()

In [39]:
clf = RandomForestRegressor(n_estimators=200, max_features='sqrt', random_state=0)
clf.fit(XTrain, yTrain)
yPred= clf.predict(XTest)
print('Root Mean Squared Error:',np.sqrt(metrics.mean_squared_error(yTest, yPred)))

Root Mean Squared Error: 3916.8499919690016
